In [11]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [12]:
#data
data=datasets.load_breast_cancer()
X,Y=data.data,data.target
n_samples,n_features=X.shape
print(n_samples,n_features)

569 30


In [13]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=18)

In [14]:
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)


X_train=torch.from_numpy(X_train.astype(np.float32))
X_test=torch.from_numpy(X_test.astype(np.float32))
Y_train=torch.from_numpy(Y_train.astype(np.float32))
Y_test=torch.from_numpy(Y_test.astype(np.float32))

Y_train=Y_train.view(Y_train.shape[0],1) #column vector
Y_test=Y_test.view(Y_test.shape[0],1)

In [15]:
#model f=wx+b
class LogisticRegression(nn.Module):
    def __init__(self,n_input_features):
        super(LogisticRegression,self).__init__()
        self.linear=nn.Linear(n_input_features,1)
    
    def forward(self,x):
        y_predict=torch.sigmoid(self.linear(x))
        return y_predict
    
model=LogisticRegression(n_features)

In [16]:
learning_rate=0.01

In [17]:
#loss and optimizer
loss=nn.BCELoss() #binary cross entropy loss
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)


In [18]:
#training loop
num_epochs=100
for epoch in range(num_epochs):
    #forward pass and loss
    Y_predict=model(X_train)
    l=loss(Y_predict,Y_train)
    #backward pass
    l.backward()
    #update weights
    optimizer.step()
    #zero gradients
    optimizer.zero_grad()
    
    if (epoch+1)%10==0:
        print(f'epoch:{epoch+1},loss={l.item()}')

epoch:10,loss=0.5608439445495605
epoch:20,loss=0.46815475821495056
epoch:30,loss=0.40910643339157104
epoch:40,loss=0.3679598867893219
epoch:50,loss=0.33744096755981445
epoch:60,loss=0.31376463174819946
epoch:70,loss=0.29476338624954224
epoch:80,loss=0.27910518646240234
epoch:90,loss=0.26592570543289185
epoch:100,loss=0.25463923811912537


In [19]:
#evaluate model
with torch.no_grad():
    Y_predicted=model(X_test)
    Y_predicted_cls=Y_predicted.round()
    acc=Y_predicted_cls.eq(Y_test).sum()/float(Y_test.shape[0])
    print(f'accuracy={acc}')

accuracy=0.9122806787490845
